In [3]:
!pip install spacy

     |████████████████████████████████| 12.4 MB 706 kB/s eta 0:00:01
     |████████████████████████████████| 105 kB 723 kB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 747 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 822 kB/s eta 0:00:01
     |████████████████████████████████| 449 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 550 kB/s eta 0:00:01
     |████████████████████████████████| 113 kB 614 kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107095 sha256=b5db092962f2bcaf1f1d3e98226c459957055bb5463e47ae1bc7b85582edc64d
  Stored in directory: /Users/khai/Library/Caches/pip/wheels/11/73/9a/f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open


In [4]:
import pandas as pd
import re
import nltk
import spacy
import string

In [30]:
#Import the dataset 
full_df = pd.read_csv("/Users/khai/Downloads/bugtest.csv", nrows=30)
full_df.head()
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
df.head()

<ipython-input-30-09531c880e78>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].astype(str)


,text
0,user-ag mozilla/4 compat msie window nt avant ...
1,beta recreat shipped-local file browser scratc...
2,moment nscssframeconstructor attributechang ca...
3,user-ag mozilla/5 window window nt en-u rv gec...
4,upload extens wrong min/maxvers lengthi error ...


In [31]:
df["lowertext"] = df["text"].str.lower() 
df["lowertext"].head()

<ipython-input-31-2f8be1036731>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lowertext"] = df["text"].str.lower()


0    user-ag mozilla/4 compat msie window nt avant ...
1    beta recreat shipped-local file browser scratc...
2    moment nscssframeconstructor attributechang ca...
3    user-ag mozilla/5 window window nt en-u rv gec...
4    upload extens wrong min/maxvers lengthi error ...
Name: lowertext, dtype: object

### REMOVE PUNCTUATION

In [32]:
PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','', PUNCT))
df["nopuntext"] = df["text"].apply(lambda text: remove_punctuation(text))
df["nopuntext"].head()

<ipython-input-32-b2308fa8cfd4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nopuntext"] = df["text"].apply(lambda text: remove_punctuation(text))


0    userag mozilla4 compat msie window nt avant br...
1    beta recreat shippedlocal file browser scratch...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 window window nt enu rv gecko2...
4    upload extens wrong minmaxvers lengthi error m...
Name: nopuntext, dtype: object

### REMOVE STOP WORDS

In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPW = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPW])
df["stopwtext"] = df["nopuntext"].apply(lambda text: remove_stopwords(text))
df["stopwtext"].head()

[nltk_data] Downloading package stopwords to /Users/khai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-33-9a1533fb3aed>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["stopwtext"] = df["nopuntext"].apply(lambda text: remove_stopwords(text))


0    userag mozilla4 compat msie window nt avant br...
1    beta recreat shippedlocal file browser scratch...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 window window nt enu rv gecko2...
4    upload extens wrong minmaxvers lengthi error m...
Name: stopwtext, dtype: object

### Removal of frequent words

In [34]:
from collections import Counter
cnt = Counter()
for text in df["stopwtext"].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(10)
FREQWS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWS])


In [35]:
df['stopftext']=df['stopwtext'].apply(lambda text: remove_freqwords(text))
df['stopftext'].head()

<ipython-input-35-ef146d7c84c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stopftext']=df['stopwtext'].apply(lambda text: remove_freqwords(text))


0    userag mozilla4 compat msie nt avant browser a...
1    beta recreat shippedlocal file browser scratch...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 nt enu rv gecko20060728 firefo...
4    upload extens wrong minmaxvers lengthi error m...
Name: stopftext, dtype: object

### Stemming

In [39]:
from nltk.stem.porter import PorterStemmer 
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])
df["stemtext"] = df["stopftext"].apply(lambda text: stem_words(text))
df["stemtext"].head()

<ipython-input-39-8d8137d3a65b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["stemtext"] = df["stopftext"].apply(lambda text: stem_words(text))


0    userag mozilla4 compat msie nt avant browser a...
1    beta recreat shippedloc file browser scratch b...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 nt enu rv gecko20060728 firefo...
4    upload exten wrong minmaxv lengthi error messa...
Name: stemtext, dtype: object

### Lemmatization

In [44]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

[nltk_data] Downloading package wordnet to /Users/khai/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [46]:
df["lemmtext"] = df["stemtext"].apply(lambda text:lemmatize_words(text))
df["lemmtext"].head()

<ipython-input-46-21fc79bb6cfe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lemmtext"] = df["stemtext"].apply(lambda text:lemmatize_words(text))


0    userag mozilla4 compat msie nt avant browser a...
1    beta recreat shippedloc file browser scratch b...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 nt enu rv gecko20060728 firefo...
4    upload exten wrong minmaxv lengthi error messa...
Name: lemmtext, dtype: object

### Removal of URLs

In [48]:
import re
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [49]:
df["no-url"] = df["lemmtext"].apply(lambda text:remove_urls(text))
df["no-url"].head()

<ipython-input-49-b8338f4dce91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["no-url"] = df["lemmtext"].apply(lambda text:remove_urls(text))


0    userag mozilla4 compat msie nt avant browser a...
1    beta recreat shippedloc file browser scratch b...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 nt enu rv gecko20060728 firefo...
4    upload exten wrong minmaxv lengthi error messa...
Name: no-url, dtype: object

### Remove HTML

In [50]:
from bs4 import BeautifulSoup
def remove_html(text):
    return BeautifulSoup(text, "lxml").text

In [51]:
df["no-html"] = df["no-url"].apply(lambda text:remove_html(text))
df["no-html"].head()

<ipython-input-51-1103b64565d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["no-html"] = df["no-url"].apply(lambda text:remove_html(text))


0    userag mozilla4 compat msie nt avant browser a...
1    beta recreat shippedloc file browser scratch b...
2    moment nscssframeconstructor attributechang ca...
3    userag mozilla5 nt enu rv gecko20060728 firefo...
4    upload exten wrong minmaxv lengthi error messa...
Name: no-html, dtype: object

### Correct Spelling

In [59]:
!pip install pyspellchecker

     |████████████████████████████████| 2.7 MB 1.7 MB/s eta 0:00:01


In [60]:
from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
    

In [61]:
df["spellcorrect"] = df["no-html"].apply(lambda text:correct_spellings(text))
df["spellcorrect"].head()

<ipython-input-61-65064dcb2879>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["spellcorrect"] = df["no-html"].apply(lambda text:correct_spellings(text))


0    user momzilla combat sie it avant browser avan...
1    beta retreat shippedloc file browser scratch b...
2    moment nscssframeconstructor attributechang ca...
3    user momzilla it end re gecko20060728 firebox ...
4    upload eaten wrong minimax length error messag...
Name: spellcorrect, dtype: object